In [ ]:
%load_ext autoreload
%autoreload 1

In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import nidaqmx

In [ ]:
plt.style.use("dark_background")

In [ ]:
from nidaqmx.stream_readers import AnalogMultiChannelReader, AnalogSingleChannelReader
from nidaqmx.stream_writers import AnalogMultiChannelWriter, AnalogSingleChannelWriter

In [ ]:
from tqdm.notebook import tqdm

In [ ]:
from nidaqmx.constants import Edge, AcquisitionType

In [ ]:
%aimport scanning_patterns

In [ ]:
extent_x = (-3, 3)
extent_y = (-3, 3)

n_x = 400
n_y = 400
raw_x, raw_y = scanning_patterns.simple_scanning_pattern(n_x, n_y, 10, 300, True)
pos_x = raw_x*((extent_x[1]-extent_x[0])/n_x) + extent_x[0]
pos_y = raw_y*((extent_y[1]-extent_y[0])/n_y) + extent_y[0]

In [ ]:
fig, ax = plt.subplots()
ax.plot(pos_x, pos_y)
ax.set_aspect(1)


# Galvo testing

In [ ]:
sample_rate = 500000.0

plane_duration = len(raw_x)/sample_rate

In [ ]:
desired_duration = 60*60*2
n_repeats = int(round(desired_duration/plane_duration))

In [ ]:
save_every = 500
n_save_subsequent = 10

In [ ]:
from math import ceil

In [ ]:
n_saves = int(ceil(n_repeats / save_every))

In [ ]:
n_samples = len(raw_x)
time = np.arange(n_samples)/sample_rate
write_signals = np.stack([pos_x, pos_y], 0)
read_signals = np.zeros((2, n_samples*n_saves*n_save_subsequent))
values_read = np.zeros((2, n_samples))
                       
with nidaqmx.Task() as write_task, nidaqmx.Task() as read_task:
    # Configure the channels
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai1:2", min_val=-5, max_val=5)
    write_task.ao_channels.add_ao_voltage_chan("Dev1/ao0:1", min_val=-5, max_val=5)
    
    # Set the timing of both to the onboard clock so that they are synchronised
    read_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                         active_edge = Edge.RISING,
                                         sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    write_task.timing.cfg_samp_clk_timing(rate=sample_rate, source="OnboardClock",
                                          active_edge = Edge.RISING,
                                          sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples)
    
    # This is necessary to synchronise reading and wrting
    read_task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/ao/StartTrigger", Edge.RISING)
    
    writer = AnalogMultiChannelWriter(write_task.out_stream)
    reader = AnalogMultiChannelReader(read_task.in_stream)
    i_save = 0
    for i_frame in tqdm(range(n_repeats)):
        # The first write has to be defined before the task starts
        writer.write_many_sample(write_signals)
        if i_frame == 0:
            read_task.start()
            write_task.start()
        reader.read_many_sample(
            values_read, number_of_samples_per_channel=n_samples,
            timeout=1)
        if (i_frame % save_every) < n_save_subsequent:
            read_signals[:, i_save*n_samples:(i_save+1)*n_samples] = values_read
            i_save +=1 

In [ ]:
from ipywidgets import interact

In [ ]:
fig, axes = plt.subplots(1, 1, sharex=True)
axes.plot(time, write_signals[0,:])
for i in range(n_saves):
    axes.plot(time, read_signals[0,i*n_samples*n_save_subsequent:(i*n_samples*n_save_subsequent+n_samples)],
             linewidth=0.5, color=(0,0,0,0.2))

In [ ]:
fig, axes = plt.subplots(1, 1, sharex=True)
plts = axes.plot(time, write_signals[0,:])

In [ ]:
ln[0].set_dash_capstyle

In [ ]:
fig, ax = plt.subplots()
i_sig = 1
ax.plot(time, write_signals[i_sig,:])

ln = ax.plot(time, read_signals[i_sig,0*n_samples:(0+1)*n_samples])
ax.autoscale(enable=False)
@interact
def browse_plot(i_plot:(0, read_signals.shape[1]//(n_samples)-1)=0):

    ln[0].set_ydata(read_signals[i_sig,i_plot*n_samples:(i_plot+1)*n_samples])

In [ ]:
fig, ax = plt.subplots()

@interact
def browse_plot(i_plot:(0, read_signals.shape[1]//(n_samples)-1)=0):
    ax.clear()
    ax.plot(time, write_signals[0,:])
    ax.plot(time, read_signals[0,i_plot*n_samples:(i_plot+1)*n_samples])
    ax.set_xlim(0,0.01)

In [ ]:
fig, axes = plt.subplots(2, 1, sharex=True)
for i in range(2):
    axes[i].plot(time, write_signals[i,:])
    axes[i].plot(time, read_signals[i,:n_samples])
    axes[i].plot(time, read_signals[i,-n_samples*2:-n_samples])

# 2p scanning

In [ ]:
extent_x = (-3, 3)
extent_y = (-3, 3)

n_x = 400
n_y = 400
raw_x, raw_y = scanning_patterns.simple_scanning_pattern(n_x, n_y, 10, 300, True)
pos_x = raw_x*((extent_x[1]-extent_x[0])/n_x) + extent_x[0]
pos_y = raw_y*((extent_y[1]-extent_y[0])/n_y) + extent_y[0]

n_bin = 10

In [ ]:
n_samples_out = len(raw_x)
n_samples_in = n_samples_out*n_bin

In [ ]:
sample_rate_out = 500000.0
plane_duration = len(raw_x)/sample_rate_out

sample_rate_in = n_bin*sample_rate_out

In [ ]:
time_in = np.arange(n_samples_in)/sample_rate_in
write_signals = np.stack([pos_x, pos_y], 0)
n_repeats = 1
read_signals = np.zeros((1, n_samples_in*n_repeats))
values_in = np.zeros((2, n_samples_in))
with nidaqmx.Task() as write_task, nidaqmx.Task() as read_task:
    # Configure the channels
    read_task.ai_channels.add_ai_voltage_chan("Dev1/ai0:1", min_val=-5, max_val=5)
    write_task.ao_channels.add_ao_voltage_chan("Dev1/ao0:1", min_val=-10, max_val=10)
    
    # Set the timing of both to the onboard clock so that they are synchronised
    read_task.timing.cfg_samp_clk_timing(rate=sample_rate_in, source="OnboardClock",
                                         active_edge = Edge.RISING,
                                         sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples_in)
    write_task.timing.cfg_samp_clk_timing(rate=sample_rate_out, source="OnboardClock",
                                          active_edge = Edge.RISING,
                                          sample_mode=AcquisitionType.CONTINUOUS, samps_per_chan=n_samples_out)
    
    # This is necessary to synchronise reading and wrting
    read_task.triggers.start_trigger.cfg_dig_edge_start_trig("/Dev1/ao/StartTrigger", Edge.RISING)
    
    writer = AnalogMultiChannelWriter(write_task.out_stream)
    reader = AnalogMultiChannelReader(read_task.in_stream)
    
    i_save = 0
    for i_frame in tqdm(range(n_repeats)):
        # The first write has to be defined before the task starts
        writer.write_many_sample(write_signals)
        if i_frame == 0:
            read_task.start()
            write_task.start()
        reader.read_many_sample(
            values_in, number_of_samples_per_channel=n_samples_in,
            timeout=1)

        
        read_signals[0, i_save*n_samples_in:(i_save+1)*n_samples_in] = values_in[0,:]
        i_save +=1
image = scanning_patterns.reconstruct_image_pattern(values_in[0, :], raw_x, raw_y, (n_y, n_x), n_bin)
im.set_data(image)

In [ ]:
plt.figure()
im = plt.imshow(np.maximum(image,0))
plt.colorbar()